In [ ]:
!pip uninstall mediapipe
# !pip install mediapipe

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import csv

In [3]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [5]:

# Function to extract landmarks from a frame
def extract_landmarks(frame):
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        
        if results.pose_landmarks is None or results.face_landmarks is None:
            return None
        
        pose_row = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in results.pose_landmarks.landmark]).flatten()
        face_row = np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in results.face_landmarks.landmark]).flatten()
        landmarks = np.concatenate((pose_row, face_row))
        return landmarks

# Function to save landmarks and labels to a CSV file
def save_to_csv(filename, landmarks, label):
    with open(filename, mode='a', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow([label] + landmarks.tolist())

# Path to your labeled video
video_path = 'D:\Videos for ML\V1.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Initialize variables
data = []
sequence_length = 10  # Adjust the sequence length as needed
sequence_buffer = []

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Extract landmarks
    landmarks = extract_landmarks(frame)
    
    if landmarks is not None:
        # Add landmarks to sequence buffer
        sequence_buffer.append(landmarks)
        
        # If the buffer is full, create a sequence
        if len(sequence_buffer) == sequence_length:
            # Concatenate landmarks from all frames in the buffer
            sequence = np.concatenate(sequence_buffer)
            
            # Append the sequence to the data list
            data.append(sequence)
            print(sequence)
            # Clear the sequence buffer
            sequence_buffer = []

cap.release()

# Convert the data list to a DataFrame
df = pd.DataFrame(data)

# Add labels to the DataFrame (assuming you have class labels for each video)
# For simplicity, you can manually set the label here
# Change 'class_name' to the label for the current video
df['label'] = 'class_name'

# Save the DataFrame to a CSV file
df.to_csv('labeled_data.csv', index=False)  # Adjust filename as needed


[ 0.08335486  0.47619551 -0.12045951 ...  0.52368534 -0.0109936
  0.        ]
[ 0.46593323  0.5318324  -0.06152707 ...  0.50843495  0.02029257
  0.        ]
[ 0.42073292  0.53965408 -0.27267492 ...  0.44757631  0.03032262
  0.        ]
